## Let's prepare and run a (pre-compiled) test-case with the sea ice model neXtSIM-DG

In this notebook you will: 
* 2.1 Load python libraries,
* 2.2 Prepare the work directory,
* 2.3 Create and edit a configuration file that set various parameters of the model,
* 2.4 Run the model for a 5-day simulation !

---
## 2.1 Load python libraries

In [1]:
#standart libraries
import os,sys
import numpy as np

# xarray
import xarray as xr

# plot
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib import animation
from IPython.display import HTML

#JUPYTER notebook magics
%matplotlib inline 

---
## 2.2 Prepare the work directory 
(where the model is going to be run)

* **Set path of the work directory:**

In [2]:
# path of the work directory:
dirwo = "/home/jovyan/work/"

* **go to this directory:**

(for now it is empty)

In [3]:
cd  $dirwo

/home/jovyan/work


* **Links to files**: We then need to run the following lines (bash script) to make link to the various files we need (model executable, forcing, mesh, initial conditions) to our current directory.

_Note:_ the first line `%%bash` indicates that the above cell is written in bash (default is python)

In [4]:
%%bash

# link the precompiled executable of the nextsimdg model to the work directory
ln -sf /usr/local/bin/nextsim .

# link the forcing files for atmospheric (ERA5) and ocean (TOPAZ)  boundary conditions to the work directory
ln -sf /data/model-forcings/25km_NH.TOPAZ4_2010-01-01_2011-01-01.nc .
ln -sf /data/model-forcings/25km_NH.ERA5_2010-01-01_2011-01-01.nc .

# link the mesh file to the work directory
ln -sf ../nextsimdg/run/25km_NH_newmask.smesh 

# link the initial conditions  to the work directory
ln -sf ../nextsimdg/run/init_25km_NH.nc .

# list files in the work directory as a check:
ls

25km_NH.ERA5_2010-01-01_2011-01-01.nc
25km_NH_newmask.smesh
25km_NH.TOPAZ4_2010-01-01_2011-01-01.nc
init_25km_NH.nc
nextsim


---
## 2.3 Create and edit a configuration file that set various parameters of the model

_Note:_ the first line of the cell below ( `%%writefile ./config_myfirsttestcase.cfg` ) will create  a file of this name and write the following text in it. If a previous file already exists with same name it will be over-written.

In [5]:
%%writefile ./config_myfirsttestcase.cfg

[model]
init_file = init_25km_NH.nc
start = 2010-01-01T00:00:00Z
stop = 2010-01-05T00:00:00Z
time_step = P0-0T00:20:00


[Modules]
Nextsim::IDiagnosticOutput = Nextsim::ConfigOutput
Nextsim::IDynamics = Nextsim::Dynamics
Nextsim::IIceThermodynamics = Nextsim::ThermoWinton
Nextsim::IAtmosphereBoundary = Nextsim::ERA5Atmosphere
Nextsim::IOceanBoundary = Nextsim::FluxConfiguredOcean


[ConfigOutput]
period = P0-0T12:00:00
start = 2010-01-01T00:00:00Z
field_names = hsnow,hice,tice,cice
filename = output_simudyn_5d.nc

[ERA5Atmosphere]
file = 25km_NH.ERA5_2010-01-01_2011-01-01.nc

[TOPAZOcean]
file = 25km_NH.TOPAZ4_2010-01-01_2011-01-01.nc

[FluxConfiguredOcean]
qio = 2.0
sss = 36.36
sst = -1.9998
mld = 10.
current_u = 0.
current_v = 0.

Writing ./config_myfirsttestcase.cfg


* In the `[model]` section, the file above indicates the dates at which the model will start and stop the simulation with the `start`and `stop`parameters. The `time_step` is also set (here it is set to 1 hour).
* The `[Module]`section is where you set which physics, thermodynamics, forcing modules you want to activate for this run.
    * We  activate the `Dynamics` module, which includes DG advection.
    * We also activate the thermodynamics based on Winton (2000 ).
    * We will use ERA5 atmospheric forcing from files prepared by Tim (file names set in the `[ERA5Atmosphere]` section).
    * We will use a constant ocean specified in the `[FluxConfiguredOcean]`section.
* The `[ConfigOutput]` section is where you set which kind of outputs you save from the simulation. 
    * Today we will save 4 variables of the sea ice state: `hsnow`,`hice`,`tice`,`cice` (snow height, ice thickness, ice temperature, ice concentration resp.) every day (the `period` parameter).

_Note:_ if you want to re-open this config file to read the parameters, just as a check, you can do:

In [6]:
%%bash
cat ./config_myfirsttestcase.cfg


[model]
init_file = init_25km_NH.nc
start = 2010-01-01T00:00:00Z
stop = 2010-01-05T00:00:00Z
time_step = P0-0T00:20:00


[Modules]
Nextsim::IDiagnosticOutput = Nextsim::ConfigOutput
Nextsim::IDynamics = Nextsim::Dynamics
Nextsim::IIceThermodynamics = Nextsim::ThermoWinton
Nextsim::IAtmosphereBoundary = Nextsim::ERA5Atmosphere
Nextsim::IOceanBoundary = Nextsim::FluxConfiguredOcean


[ConfigOutput]
period = P0-0T12:00:00
start = 2010-01-01T00:00:00Z
field_names = hsnow,hice,tice,cice
filename = output_simudyn_5d.nc

[ERA5Atmosphere]
file = 25km_NH.ERA5_2010-01-01_2011-01-01.nc

[TOPAZOcean]
file = 25km_NH.TOPAZ4_2010-01-01_2011-01-01.nc

[FluxConfiguredOcean]
qio = 2.0
sss = 36.36
sst = -1.9998
mld = 10.
current_u = 0.
current_v = 0.


---
## 2.4 Run the model !

We are now all set to run our first simulation with the NeXtSIM-DG model! 

To do so, simply run the `nextsim` executable with the config file as it is done in the cell below. 

It should take about 5 minutes to run our test-case (5-day simulation). When it is done, you will see the list of the files in the directory. A new file should have been created: `output_simudyn_5d.nc` that contains the evolution of sea ice simulated by the model over the period we just set.

In [7]:
%%bash

pwd 

echo "------------------------------------"
echo "Yay! NeXtSIM-DG is running !........"
echo "------------------------------------"

./nextsim --config-file config_myfirsttestcase.cfg "$@" > stdout.asc

ls -lrt

/home/jovyan/work
------------------------------------
Yay! NeXtSIM-DG is running !........
------------------------------------
total 12468
lrwxrwxrwx 1 jovyan pr-sasip      22 Jun 26 20:01 nextsim -> /usr/local/bin/nextsim
lrwxrwxrwx 1 jovyan pr-sasip      60 Jun 26 20:01 25km_NH.TOPAZ4_2010-01-01_2011-01-01.nc -> /data/model-forcings/25km_NH.TOPAZ4_2010-01-01_2011-01-01.nc
lrwxrwxrwx 1 jovyan pr-sasip      38 Jun 26 20:01 25km_NH_newmask.smesh -> ../nextsimdg/run/25km_NH_newmask.smesh
lrwxrwxrwx 1 jovyan pr-sasip      58 Jun 26 20:01 25km_NH.ERA5_2010-01-01_2011-01-01.nc -> /data/model-forcings/25km_NH.ERA5_2010-01-01_2011-01-01.nc
lrwxrwxrwx 1 jovyan pr-sasip      32 Jun 26 20:01 init_25km_NH.nc -> ../nextsimdg/run/init_25km_NH.nc
-rw-r--r-- 1 jovyan pr-sasip     743 Jun 26 20:01 config_myfirsttestcase.cfg
-rw-r--r-- 1 jovyan pr-sasip 2076697 Jun 26 20:01 landmask1.00000.vtk
-rw-r--r-- 1 jovyan pr-sasip 2076697 Jun 26 20:01 boundary1.00000.vtk
-rw-r--r-- 1 jovyan pr-sasip   24682 J

---
## 2.5 What's next ?

--> You can now have a look at your first model outputs with the [next notebook (3-Plot-model-outputs.ipynb)](3-Plot-model-outputs.ipynb) 